In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Function tool


### Install adk

In [ ]:
%pip install --upgrade --quiet google-adk

In [ ]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[1]
print(MODEL_NAME)

gemini-2.5-pro-preview-03-25


In [ ]:
# @title Authentication to access to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.


In [ ]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## adk with tools

In [ ]:
#@title Tool Define - Exchange rate

def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest", )->dict:
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "KRW" (KRW).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

#------------------------------------------------------------------------

# Tool Test
get_exchange_rate(currency_from="USD", currency_to="KRW")


{'amount': 1.0, 'base': 'USD', 'date': '2025-04-25', 'rates': {'KRW': 1439.49}}

In [ ]:
#@title Tool define - get city coordinates

def get_city_coordinates(
    city_name:str = "Gwacheon-si")-> dict:

    """
    Gets the latitude and longitude of a given city.

    Args:
        city_name: The name of the city.

    Returns:
        A dict containing the latitude and longitude if found, otherwise None.
    """

    import geopy
    from geopy.geocoders import Nominatim

    try:
        geolocator = Nominatim(user_agent="geo_app")  # Replace with your app name
        location = geolocator.geocode(city_name)
        if location:
            response = {
                "latitude": location.latitude,
                "longitude": location.longitude
            }

            return response
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

#-------------------------------------------------------------

# Example usage:
city_name = "Gwacheon-si"  # Or the actual city name you want to use
coordinates = get_city_coordinates(city_name)

if coordinates:
    print(f"coordinates: {coordinates}")
else:
    print(f"Could not find coordinates for {city_name}")

coordinates: {'latitude': 37.4289037, 'longitude': 126.9881655}


In [ ]:
from google.adk.tools import FunctionTool

exchange_rate_tool = FunctionTool(func=get_exchange_rate)
city_coordinates_tool = FunctionTool(func=get_city_coordinates)


In [ ]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using funtion tool when needed.

"""

# https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
generate_content_config= types.GenerateContentConfig(
  temperature=0.5,
  max_output_tokens=1024,
)

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.agents.LlmAgent
agent = LlmAgent(
      name="Helpful_assistant",
      model=MODEL_NAME,
      instruction=PROMPT,
      generate_content_config = generate_content_config,
      description="An assistant providing answer to the user's question.",
      tools=[exchange_rate_tool, city_coordinates_tool ] # Use Function Tool
)

In [ ]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

def call_agent(query:str,
               user_id:str = "shins",
               session_id:str ="1234",
               ):

  app_name = "Assistant"

  # https://google.github.io/adk-docs/sessions/
  session_service = InMemorySessionService()
  session = session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

  # Runner : An in-memory Runner for testing and development.
  # https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner
  runner = Runner(agent=agent,
                  app_name=app_name,
                  session_service=session_service)

  # https://googleapis.github.io/python-genai/genai.html#genai.types.Content
  content = types.Content(role='user', parts=[types.Part(text=query)])

  # https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.runners.Runner.run
  events = runner.run(user_id=user_id,
                      session_id=session_id,
                      new_message=content)

  for event in events:
      print(event)

      # if event.is_final_response():
      #     final_response = event.content.parts[0]
      #     print("Agent Response: \n", final_response)
      #     return final_response


In [ ]:
USER_ID = "shins"
SESSION_ID="1234"

QUERY = "What is the exchange rate from US dollars to Korean currency recently ?"
QUERY = "What's the Coordinates of Seoul?"
# QUERY = "What is the exchange rate from US dollars to Korean currency recently and What's the Coordinates of Seoul?"

call_agent(user_id=USER_ID,
           session_id=SESSION_ID,
           query=QUERY)

content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id='adk-7fe9f5ab-77b0-4222-ba18-0314eedcdfe4', args={'city_name': 'Seoul'}, name='get_city_coordinates'), function_response=None, inline_data=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None invocation_id='e-4bcdb06f-71fd-429a-ad07-2e25891a41b5' author='Helpful_assistant' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=set() branch=None id='aryZdyj9' timestamp=1745744216.382924
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=FunctionResponse(id='adk-7fe9f5ab-77b0-4222-ba18-0314eedcdfe4'

content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='The coordinates for Seoul are approximately:\n*   **Latitude:** 37.57\n*   **Longitude:** 126.98')], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None invocation_id='e-4bcdb06f-71fd-429a-ad07-2e25891a41b5' author='Helpful_assistant' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=None branch=None id='5Cv1uPIN' timestamp=1745744218.862987


## End of notebook